In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [2]:
from atlas_rag.kg_construction.triple_extraction import KnowledgeGraphExtractor
from atlas_rag.kg_construction.triple_config import ProcessingConfig
from atlas_rag.llm_generator import LLMGenerator
from openai import OpenAI
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from transformers import pipeline
from configparser import ConfigParser
# Load OpenRouter API key from config file
config = ConfigParser()
config.read('config.ini')
# model_name = "meta-llama/Llama-3.3-70B-Instruct"
# connection = AIProjectClient(
#     endpoint=config["urls"]["AZURE_URL"],
#     credential=DefaultAzureCredential(),
# )
# client = connection.inference.get_azure_openai_client(api_version="2024-12-01-preview")
client = OpenAI(base_url="http://10.67.74.45:8001/v1/", api_key="sk-6aP9xT2mQv1sRb8Zc4NyUe0Lf7Hk3Wd5")
triple_generator = LLMGenerator(client=client, model_name="qwen3-30B-A3B-Instruct")

# model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"
# model_name = "meta-llama/Llama-3.2-3B-Instruct"
# client = pipeline(
#     "text-generation",
#     model=model_name,
#     device_map="auto",
# )
filename_pattern = 'plantstation_100'
output_directory = f'./output/plantstation_kg'
# triple_generator = LLMGenerator(client, model_name=model_name)
model_name = "qwen3-30B-A3B-Instruct"

D:\conda\envs\atlas-rag\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Initialized LLMGenerator with inference_type='api', backend='openai'


In [3]:
kg_extraction_config = ProcessingConfig(
      model_path=model_name,
      # data_directory=f'example/example_data/multilingual_data/{filename_pattern}',
      data_directory=f'json_data/测点/',
      filename_pattern=filename_pattern,
      batch_size_triple=16,
      batch_size_concept=16,
      output_directory=f"{output_directory}",
      max_new_tokens=2048,
      max_workers=3,
      remove_doc_spaces=True, # For removing duplicated spaces in the document text
      include_concept=False, # Whether to include concept nodes and edges in the knowledge graph
      triple_extraction_prompt_path='prompts/custom_prompt.json',
      triple_extraction_schema_path='prompts/custom_schema.json',
      record=True, # Whether to record the results in a JSON file
)
kg_extractor = KnowledgeGraphExtractor(model=triple_generator, config=kg_extraction_config)

import os
data_directory=f'json_data/测点/'
data_path = os.path.join(data_directory, filename_pattern + ".json")
print("目标文件路径：", data_path)  # 应输出正确的 JSON 文件路径
print("路径是否存在：", os.path.exists(data_path))  # 应返回 True

Using custom kg extraction prompt:
{'zh-CN': {'system': '你是一名专业的工业电力设备测点信息提取专家，专注于处理KKS码相关的设备测点文本，能够准确识别各类工业术语，并且会原样提取文本中明确标注的`null`值，确保提取的信息完整无遗漏。', 'triple_extraction': '你是知识图谱构建专家。\n你的任务是从提供的工业KKS测点文本中提取所有真实有效的事实信息，并严格以***JSON数组***的形式返回知识图谱三元组。\n\n### 输出格式\n- 输出必须是一个**JSON数组**。\n- 数组中的每个元素必须是一个**JSON对象**，且只能包含三个非空关键字段：\n  - "subject"：核心实体，优先使用原文中的精确表述（例如：“测点SGC.SHIDONGKOU_2.20MBA26CT121BXQ01”、“设备AA101”、“MBA主系统”、“10 号机组”、“55 区域”、“CHANGXING_1 电厂”）。\n  - "relation"：简洁准确的描述性短语或动词，用于体现主体与客体的关联关系（例如：“的核心KKS码为”、“设备类型是”、“设备序列号是”、“位于”、“应用于”、“包含”、“部署”、“划分”、“无效代码标识为”、“功能描述为”、“测量值单位为”、“用于监测”）。\n  - "object"：与主体存在关联的实体、属性或值，工业术语需保持原文一致，文本中明确标注的`null`需原样作为字符串“null”填入（例如：“CT”、“121”、“ CHANGXING_1 电厂“、”10 号机组“、”HAD 主系统“、”55 区域“、”设备AA101”、“BXQ01”、“null”、“%”）。\n\n### 约束条件\n- **除JSON输出外，不得包含任何其他文本**（如解释、注释、格式说明、换行提示等均不允许出现）。\n- 必须提取**所有可能且相关的三元组**，不得遗漏测点、设备、主系统、电厂机组的任何关键信息，包括文本中的`null`值。\n- 三个关键字段必须全部存在，且所有字段的值均为非空字符串（文本中的`null`需填写为“null”，不得省略或替换为空字符串）。\n- 工业专业术语（如KKS码、设备类型、机组编号、主系统名称等）必须严格保留原文表述，不得随意篡

In [4]:
kg_extractor.run_extraction()

Found data files: ['plantstation_100.json']


Generating train split: 1 examples [00:00, 26.78 examples/s]


Processing shard 1/1 (texts 0-0 of 1, 1 documents)
Generated 5 chunks for shard 1/1
Model: qwen3-30B-A3B-Instruct


100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:30<00:00, 30.69s/it]

Item 17 is a duplicate triple: {'subject': 'CHANGXING_1 电厂', 'relation': '包含', 'object': '10 号机组'}
Item 51 is a duplicate triple: {'subject': 'HEFEI_1 电厂', 'relation': '包含', 'object': '01 号机组'}
Item 52 is a duplicate triple: {'subject': '01 号机组', 'relation': '配置', 'object': 'LAC 主系统'}
Item 53 is a duplicate triple: {'subject': 'LAC 主系统', 'relation': '划分有', 'object': '20 区域'}
Item 57 missing required keys: {'object', 'relation'}. Problematic item: {'subject': '测点SGC'}
Item 62 missing required keys: {'object', 'relation'}. Problematic item: {'subject': '测点SGC.CHANGXING_1.10CJA00DT100STORE'}
Item 58 missing required keys: {'object', 'relation'}. Problematic item: {'subject': '测点SGC.FENGQUAN_1.10MBV10AH001ZV6'}
Item 42 is a duplicate triple: {'subject': '02 号机组', 'relation': '配置', 'object': 'PAC 主系统'}
Item 43 is a duplicate triple: {'subject': 'PAC 主系统', 'relation': '划分', 'object': '01 区域'}
Item 44 is a duplicate triple: {'subject': '01 区域', 'relation': '部署', 'object': '设备 UC020'}
Item 45 

In [5]:
kg_extractor.convert_json_to_csv()

Loading data from the json files
Number of files:  1


100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 167.40it/s]

Processing file for file ids:  qwen3-30B-A3B-Instruct_plantstation_100_output_20251229124932_1_in_1.json


In [6]:
kg_extractor.generate_concept_csv_temp(language='zh-CN')

all_batches 12


Shard_0:   0%|                                                                                                  | 0/12 [00:00<?, ?it/s]2025-12-29 12:54:37,935 - INFO - Usage log: Node STORE, completion_usage: {'completion_tokens': 19, 'prompt_tokens': 469, 'total_tokens': 488, 'completion_tokens_details': None, 'prompt_tokens_details': None, 'time': 0.300675630569458}
2025-12-29 12:54:37,936 - INFO - Usage log: Node MAA 主系统, completion_usage: {'completion_tokens': 19, 'prompt_tokens': 471, 'total_tokens': 490, 'completion_tokens_details': None, 'prompt_tokens_details': None, 'time': 0.36389851570129395}
2025-12-29 12:54:37,937 - INFO - Usage log: Node °, completion_usage: {'completion_tokens': 11, 'prompt_tokens': 472, 'total_tokens': 483, 'completion_tokens_details': None, 'prompt_tokens_details': None, 'time': 0.21654605865478516}
2025-12-29 12:54:37,938 - INFO - Usage log: Node CHUZHOU_1 电厂, completion_usage: {'completion_tokens': 20, 'prompt_tokens': 483, 'total_tokens': 503, 'comp

Number of unique conceptualized nodes: 211
Number of unique conceptualized events: 0
Number of unique conceptualized entities: 144
Number of unique conceptualized relations: 69


In [7]:
kg_extractor.create_concept_csv()

Loading concepts...


171it [00:00, 114499.68it/s]


Loading concepts done.
Relation to concepts: 18
Node to concepts: 153
Processing triple nodes...


153it [00:00, 34147.21it/s]


Processing concept nodes...


100%|████████████████████████████████████████████████████████████████████████████████████████████| 144/144 [00:00<00:00, 289262.34it/s]


Processing triple edges...


241it [00:00, 96813.26it/s]


In [8]:
from sentence_transformers import SentenceTransformer
# from atlas_rag.vectorstore.embedding_model import SentenceEmbedding
from atlas_rag.vectorstore.embedding_model import EmbeddingAPI
# use sentence embedding if you want to use sentence transformer
# use NvEmbed if you want to use NvEmbed-v2 model
# sentence_model = SentenceTransformer('all-MiniLM-L6-v2')
# sentence_encoder = SentenceEmbedding(sentence_model)
sentence_encoder = EmbeddingAPI(client, model_name="qwen3-embedding-8B")

In [9]:
# add numeric id to the csv so that we can use vector indices
kg_extractor.add_numeric_id()

# compute embedding
kg_extractor.compute_kg_embedding(sentence_encoder) # default encoder_model_name="all-MiniLM-L12-v2", only compute all embeddings except any concept related embeddings
# kg_extractor.compute_embedding(encoder_model_name="all-MiniLM-L12-v2")
# kg_extractor.compute_embedding(encoder_model_name="nvidia/NV-Embed-v2")

# create faiss index
kg_extractor.create_faiss_index(faiss_gpu=False) # default index_type="HNSW,Flat", other options: "IVF65536_HNSW32,Flat" for large KG
# kg_extractor.create_faiss_index(index_type="HNSW,Flat")
# kg_extractor.create_faiss_index(index_type="IVF65536_HNSW32,Flat")

['name:ID', 'type', 'concepts', 'synsets', ':LABEL']


Adding numeric ID: 153it [00:00, 153670.62it/s]


[':START_ID', ':END_ID', 'relation', 'concepts', 'synsets', ':TYPE']


Adding numeric ID: 241it [00:00, 121057.16it/s]


['text_id:ID', 'original_text', ':LABEL']


Adding numeric ID: 5it [00:00, 2009.15it/s]


Processed chunk 0 (100000 rows)
Total number of embeddings: 153
Conversion complete!
Processed chunk 0 (100000 rows)
Total number of embeddings: 5
Conversion complete!


2025-12-29 12:55:37,060 - INFO - Stopped loading due to: No data left in file
2025-12-29 12:55:37,062 - INFO - Adding time: 0.02 seconds
2025-12-29 12:55:37,070 - INFO - Stopped loading due to: No data left in file
2025-12-29 12:55:37,071 - INFO - Adding time: 0.00 seconds
2025-12-29 12:55:37,090 - INFO - Stopped loading due to: No data left in file
2025-12-29 12:55:37,091 - INFO - Adding time: 0.01 seconds


Processed chunk 0 (100000 rows)
Total number of embeddings: 241
Conversion complete!
Dimension: 4096
Dimension: 4096
Dimension: 4096
